In [1]:

import pandas as pd
import plotly.express as px

df = pd.read_csv('nuclear_data/impacts-of-energy-sources.csv')

df = df.drop(['Agricultural land use', 'Urban land use', 'Death rates', 'Year'], axis=1)

df = df.rename(columns={'Greenhouse gas emissions': 'Greenhouse gas emissions (kg CO2 eq.)', 
                        'Freshwater eutrophication': 'Freshwater eutrophication (kg P eq.)', 
                        'Ionising radiation':'Ionising radiation (kBq 235U eq)', 
                        'Water use':'Water use (m3)',
                        'Metal and mineral requirements':'Metal and mineral requirements (kg Sb eq.)',
                        'Non-carcinogenic toxicity':'Non-carcinogenic toxicity (CTUh)',
                        'Carcinogenic toxicity':'Carcinogenic toxicity (CTUh)',
                        'Total land use':'Total land use (points)',
                        'Uranium':'Uranium (kg Sb eq.)'})

df['Entity'] = df['Entity'].replace('Coal with carbon capture (CCS)', 'Coal')

df['Entity'] = df['Entity'].replace('Concentrating solar (tower)', 'Concentrating Solar')
df['Entity'] = df['Entity'].replace('Concentrating solar (trough)', 'Concentrating Solar')

df['Entity'] = df['Entity'].replace('Solar PV, cadmium (roof)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, cadmium (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (roof)', 'Solar PV')

df['Entity'] = df['Entity'].replace('Gas with carbon capture (CCS)', 'Gas')

df['Entity'] = df['Entity'].replace('Offshore wind', 'Wind')
df['Entity'] = df['Entity'].replace('Onshore wind', 'Wind')

df_combined = df.drop(columns=['Aluminium', 'Chromium', 'Cobalt', 'Copper', 'Manganese', 'Molybdenum', 'Nickel', 'Silicon', 'Zinc'])
df_combined['Human toxicity (CTUh)'] = df_combined['Non-carcinogenic toxicity (CTUh)'] + df_combined['Carcinogenic toxicity (CTUh)']
df_combined = df_combined.drop(columns=['Non-carcinogenic toxicity (CTUh)', 'Carcinogenic toxicity (CTUh)'])
df_combined = df_combined[['Entity', 'Greenhouse gas emissions (kg CO2 eq.)', 
                           'Freshwater eutrophication (kg P eq.)', 
                           'Ionising radiation (kBq 235U eq)', 
                           'Water use (m3)', 
                           'Total land use (points)', 
                           'Human toxicity (CTUh)', 
                           'Metal and mineral requirements (kg Sb eq.)', 
                           'Uranium (kg Sb eq.)']]

df_combined = df_combined.groupby('Entity', as_index=False).sum()

for column in df_combined.columns[1:]:
    fig = px.bar(df_combined, 
                 x='Entity', 
                 y=column,
                 title=f'{column} by Energy Source per the delivery of 1 kWh of electricity (2021)',
                 labels={'Entity': 'Energy Source', column: column},
                 color='Entity')
    
    fig.update_layout(
        xaxis_title='Energy Source',
        yaxis_title=column,
        height=600,
        width=1000,
        title_font=dict(size=18),
        title_x=0.5
    )
    
    fig.update_xaxes(tickangle=45)
    
    fig.show()
